**Context**

Reduction of child mortality is reflected in several of the United Nations' Sustainable Development Goals and is a key indicator of human progress.
The UN expects that by 2030, countries end preventable deaths of newborns and children under 5 years of age, with all countries aiming to reduce under‑5 mortality to at least as low as 25 per 1,000 live births.

Parallel to notion of child mortality is of course maternal mortality, which accounts for 295 000 deaths during and following pregnancy and childbirth (as of 2017). The vast majority of these deaths (94%) occurred in low-resource settings, and most could have been prevented.

In light of what was mentioned above, Cardiotocograms (CTGs) are a simple and cost accessible option to assess fetal health, allowing healthcare professionals to take action in order to prevent child and maternal mortality. The equipment itself works by sending ultrasound pulses and reading its response, thus shedding light on fetal heart rate (FHR), fetal movements, uterine contractions and more.

**Data**

This dataset contains 2126 records of features extracted from Cardiotocogram exams, which were then classified by three expert obstetritians into 3 classes:

Normal
Suspect
Pathological


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pandas.plotting import scatter_matrix
# For Hyper-parameter Tuning the model
from sklearn.model_selection import GridSearchCV

# For checking Model Performance
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve

import warnings
warnings.simplefilter(action="ignore")

from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
df=pd.read_csv('/kaggle/input/fetal-health-classification/fetal_health.csv')
df.head()

In [ ]:
features=df.columns
features

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:
corr_matrix=df.corr()
corr_matrix['fetal_health'].sort_values(ascending=False)

correlation coefficient ranges from -1 to 1.

* close to 1- strong positive correlation
* close to -1 - strong negative correlation
* close to 0- no correlation

if we observe above correlation matrix,some attributes(fetal_movement,histogram_min, light_decelerations,histogram_number_of_zeroes,
histogram_number_of_peaks, histogram_max,
histogram_width) have coeffients close to zero means that there is no linear correlation.

and histogram_mean,histogram_mode shows same correlation

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix)
plt.show()

In [ ]:
df = df.drop(['histogram_min','histogram_median','histogram_mode'], axis=1)

In [ ]:
sns.countplot(df['fetal_health'])

conclusion: fetal_health is normally distributed

In [ ]:
print("The Number of Samples in the dataset: ", len(df))
print('Class 1:', round(df['fetal_health'].value_counts()[1]/len(df) * 100, 2), '% of the dataset')

print('Class 2 :', round(df['fetal_health'].value_counts()[2]/len(df) * 100, 2), '% of the dataset')
print('Class 3 :', round(df['fetal_health'].value_counts()[3]/len(df) * 100, 2), '% of the dataset')


In [ ]:
print("Skewness: %f" %df['fetal_health'].skew())

highly skewed

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(['fetal_health'], axis=1)
y = df['fetal_health']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.30, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
cv_method = StratifiedKFold(n_splits=3)
param={'criterion':('gini', 'entropy'),'min_samples_split': (2, 6, 20),'min_samples_leaf': (1, 4, 16),'n_estimators' :(100,150, 200, 250)}
clf = GridSearchCV(rf, param,cv=cv_method,scoring="accuracy")
clf.fit(X_train,Y_train)

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_

In [ ]:
model = RandomForestClassifier(criterion= 'gini',min_samples_leaf= 1,min_samples_split=2,n_estimators= 150)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

In [ ]:
pred = model.predict(X_test)
print("Classification Report")
print(classification_report(Y_test, pred))
ax= plt.subplot()
sns.heatmap(confusion_matrix(Y_test, pred), annot=True, ax = ax);

# labels, title and ticks
ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")
ax.set_title("Confusion Matrix")
ax.xaxis.set_ticklabels(["Normal", "Suspect", "Pathological"])

# Why I used Random Forest Classifier?
* perform implicit feature selection
* methods for balancing error in unbalanced dataset(here, our target is unbalanced and we didn't use any sampling technique)